## എംബെഡിംഗുകൾ

മുൻ ഉദാഹരണത്തിൽ, നാം ഉയർന്ന-ഡൈമെൻഷണൽ ബാഗ്-ഓഫ്-വേർഡ് വെക്ടറുകളുമായി പ്രവർത്തിച്ചിരുന്നു, അവയുടെ നീളം `vocab_size` ആയിരുന്നു, കൂടാതെ നാം താഴ്ന്ന-ഡൈമെൻഷണൽ പൊസിഷണൽ പ്രതിനിധാന വെക്ടറുകളിൽ നിന്ന് സ്പാർസ് വൺ-ഹോട്ട് പ്രതിനിധാനത്തിലേക്ക് വ്യക്തമായി മാറ്റം നടത്തുകയായിരുന്നു. ഈ വൺ-ഹോട്ട് പ്രതിനിധാനം മെമ്മറി കാര്യക്ഷമമല്ല, കൂടാതെ ഓരോ വാക്കും പരസ്പരം സ്വതന്ത്രമായി പരിഗണിക്കപ്പെടുന്നു, അഥവാ വൺ-ഹോട്ട് എൻകോഡുചെയ്ത വെക്ടറുകൾ വാക്കുകൾക്കിടയിലെ സാംസാരിക സമാനതയെ പ്രകടിപ്പിക്കുന്നില്ല.

ഈ യൂണിറ്റിൽ, നാം **News AG** ഡാറ്റാസെറ്റിനെ തുടർന്നു പരിശോധിക്കും. തുടങ്ങാൻ, ഡാറ്റ ലോഡ് ചെയ്ത് മുൻ നോട്ട്‌ബുക്കിൽ നിന്നുള്ള ചില നിർവചനങ്ങൾ നേടാം.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## എമ്പെഡ്ഡിംഗ് എന്താണ്?

**എമ്പെഡ്ഡിംഗ്** എന്ന ആശയം വാക്കുകളെ കുറവ്-ഡൈമെൻഷണൽ സാന്ദ്ര വെക്ടറുകളായി പ്രതിനിധീകരിക്കുന്നതാണ്, അവ വാക്കിന്റെ അർത്ഥപരമായ അർത്ഥം ഏതോ വിധത്തിൽ പ്രതിഫലിപ്പിക്കുന്നു. നാം പിന്നീട് അർത്ഥവത്തായ വാക്ക് എമ്പെഡ്ഡിംഗുകൾ എങ്ങനെ നിർമ്മിക്കാമെന്ന് ചർച്ച ചെയ്യും, പക്ഷേ ഇപ്പോൾ എമ്പെഡ്ഡിംഗുകളെ വാക്ക് വെക്ടറിന്റെ ഡൈമെൻഷൻ കുറയ്ക്കാനുള്ള ഒരു മാർഗമായി മാത്രം കരുതാം.

അതിനാൽ, എമ്പെഡ്ഡിംഗ് ലെയർ ഒരു വാക്ക് ഇൻപുട്ടായി സ്വീകരിച്ച്, നിശ്ചിത `embedding_size` ഉള്ള ഔട്ട്പുട്ട് വെക്ടർ ഉത്പാദിപ്പിക്കും. ഒരു അർത്ഥത്തിൽ, ഇത് `Linear` ലെയറിനോട് വളരെ സമാനമാണ്, പക്ഷേ ഒന്ന്-ഹോട്ട് എൻകോഡുചെയ്ത വെക്ടർ സ്വീകരിക്കുന്നതിന് പകരം, വാക്കിന്റെ നമ്പർ ഇൻപുട്ടായി സ്വീകരിക്കാൻ കഴിയും.

നമ്മുടെ നെറ്റ്‌വർക്കിലെ ആദ്യ ലെയറായി എമ്പെഡ്ഡിംഗ് ലെയർ ഉപയോഗിച്ച്, നാം ബാഗ്-ഓഫ്-വേർഡ്സ് മോഡലിൽ നിന്ന് **എമ്പെഡ്ഡിംഗ് ബാഗ്** മോഡലിലേക്ക് മാറാം, ഇവിടെ ആദ്യം നമ്മുടെ ടെക്സ്റ്റിലെ ഓരോ വാക്കും അനുയോജ്യമായ എമ്പെഡ്ഡിങ്ങിലേക്ക് മാറ്റുകയും, പിന്നീട് ആ എമ്പെഡ്ഡിങ്ങുകളുടെ മേൽ `sum`, `average` അല്ലെങ്കിൽ `max` പോലുള്ള ഏതെങ്കിലും സംഗ്രഹ ഫംഗ്ഷൻ കണക്കാക്കുകയും ചെയ്യും.

![അഞ്ച് സീക്വൻസ് വാക്കുകൾക്കുള്ള എമ്പെഡ്ഡിംഗ് ക്ലാസിഫയർ കാണിക്കുന്ന ചിത്രം.](../../../../../translated_images/ml/embedding-classifier-example.b77f021a7ee67eee.webp)

നമ്മുടെ ക്ലാസിഫയർ ന്യൂറൽ നെറ്റ്‌വർക്ക് എമ്പെഡ്ഡിംഗ് ലെയറോടെ ആരംഭിച്ച്, തുടർന്ന് അഗ്രിഗേഷൻ ലെയർ, അതിന്റെ മുകളിൽ ലീനിയർ ക്ലാസിഫയർ എന്നിവയാകും:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### വ്യത്യസ്ത സീക്വൻസ് വലുപ്പം കൈകാര്യം ചെയ്യൽ

ഈ ആർക്കിടെക്ചറിന്റെ ഫലമായി, നമ്മുടെ നെറ്റ്‌വർക്കിലേക്ക് മിനിബാച്ചുകൾ ഒരു പ്രത്യേക രീതിയിൽ സൃഷ്ടിക്കേണ്ടിവരും. മുമ്പത്തെ യൂണിറ്റിൽ, ബാഗ്-ഓഫ്-വേർഡ്സ് ഉപയോഗിക്കുമ്പോൾ, മിനിബാച്ചിലെ എല്ലാ BoW ടെൻസറുകളും വാസ്തവത്തിൽ ഉള്ള ടെക്സ്റ്റ് സീക്വൻസിന്റെ നീളത്തെ ആശ്രയിക്കാതെ സമാനമായ വലുപ്പമുള്ള `vocab_size` ആയിരുന്നു. വാക്ക് എംബെഡിംഗ്സിലേക്ക് മാറുമ്പോൾ, ഓരോ ടെക്സ്റ്റ് സാമ്പിളിലും വ്യത്യസ്തമായ വാക്കുകളുടെ എണ്ണം ഉണ്ടാകും, ആ സാമ്പിളുകൾ മിനിബാച്ചുകളായി ചേർക്കുമ്പോൾ പാഡിംഗ് പ്രയോഗിക്കേണ്ടിവരും.

ഇത് ഡാറ്റാസോഴ്സിന് `collate_fn` ഫംഗ്ഷൻ നൽകുന്ന സമാന സാങ്കേതിക വിദ്യ ഉപയോഗിച്ച് ചെയ്യാം:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### എംബെഡ്ഡിംഗ് ക്ലാസിഫയർ പരിശീലനം

ഇപ്പോൾ നമുക്ക് ശരിയായ ഡാറ്റലോഡർ നിർവചിച്ചിരിക്കുന്നു, മുൻ യൂണിറ്റിൽ നിർവചിച്ച പരിശീലന ഫംഗ്ഷൻ ഉപയോഗിച്ച് മോഡൽ പരിശീലിപ്പിക്കാം:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **കുറിപ്പ്**: സമയപരിധി കണക്കിലെടുത്ത് ഇവിടെ നാം 25,000 റെക്കോർഡുകൾക്ക് മാത്രമേ പരിശീലനം നടത്തുകയുള്ളൂ (ഒരു പൂർണ്ണ എപ്പോക്കിന് താഴെ), പക്ഷേ നിങ്ങൾക്ക് പരിശീലനം തുടർന്നു നടത്താനും, പല എപ്പോക്കുകൾക്കായി പരിശീലനം നടത്താനുള്ള ഫംഗ്ഷൻ എഴുതാനും, ഉയർന്ന കൃത്യത നേടാൻ ലേണിംഗ് റേറ്റ് പാരാമീറ്ററിൽ പരീക്ഷണങ്ങൾ നടത്താനും കഴിയും. ഏകദേശം 90% കൃത്യത നേടാൻ നിങ്ങൾക്ക് സാധിക്കണം.


### EmbeddingBag ലെയർ ಮತ್ತು വ്യത്യസ്ത നീളമുള്ള സീക്വൻസ് പ്രതിനിധാനം

മുൻവർഷത്തെ ആർക്കിടെക്ചറിൽ, മിനിബാച്ചിൽ ഫിറ്റ് ചെയ്യാൻ എല്ലാ സീക്വൻസുകളും ഒരേ നീളത്തിലേക്ക് പാഡ് ചെയ്യേണ്ടിവന്നു. വ്യത്യസ്ത നീളമുള്ള സീക്വൻസുകൾ പ്രതിനിധാനം ചെയ്യാനുള്ള ഏറ്റവും കാര്യക്ഷമമായ മാർഗം ഇത് അല്ല - മറ്റൊരു സമീപനം **ഓഫ്സെറ്റ്** വെക്ടർ ഉപയോഗിക്കുകയാണ്, ഇത് ഒരു വലിയ വെക്ടറിൽ സൂക്ഷിച്ചിരിക്കുന്ന എല്ലാ സീക്വൻസുകളുടെ ഓഫ്സെറ്റുകൾ സൂക്ഷിക്കും.

![ഓഫ്സെറ്റ് സീക്വൻസ് പ്രതിനിധാനം കാണിക്കുന്ന ചിത്രം](../../../../../translated_images/ml/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **Note**: മുകളിൽ കാണുന്ന ചിത്രത്തിൽ, ഒരു അക്ഷരങ്ങളുടെ സീക്വൻസ് കാണിക്കുന്നു, എന്നാൽ നമ്മുടെ ഉദാഹരണത്തിൽ നാം വാക്കുകളുടെ സീക്വൻസുകളുമായി പ്രവർത്തിക്കുന്നു. എന്നിരുന്നാലും, ഓഫ്സെറ്റ് വെക്ടർ ഉപയോഗിച്ച് സീക്വൻസുകൾ പ്രതിനിധാനം ചെയ്യാനുള്ള പൊതുവായ സിദ്ധാന്തം അതേപോലെ തുടരുന്നു.

ഓഫ്സെറ്റ് പ്രതിനിധാനത്തോടെ പ്രവർത്തിക്കാൻ, നാം [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html) ലെയർ ഉപയോഗിക്കുന്നു. ഇത് `Embedding` പോലെയാണ്, പക്ഷേ ഇത് ഇൻപുട്ടായി ഉള്ളടക്ക വെക്ടറും ഓഫ്സെറ്റ് വെക്ടറും സ്വീകരിക്കുന്നു, കൂടാതെ ഇത് ശരാശരി ലെയർ ഉൾക്കൊള്ളുന്നു, അത് `mean`, `sum` അല്ലെങ്കിൽ `max` ആകാം.

ഇവിടെ `EmbeddingBag` ഉപയോഗിക്കുന്ന മാറ്റം വരുത്തിയ നെറ്റ്‌വർക്ക് കാണിക്കുന്നു:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

ട്രെയിനിംഗിനായി ഡാറ്റാസെറ്റ് തയ്യാറാക്കാൻ, ഓഫ്‌സെറ്റ് വെക്ടർ തയ്യാറാക്കുന്ന ഒരു കൺവർഷൻ ഫംഗ്ഷൻ നൽകേണ്ടതുണ്ട്:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

കുറിപ്പ്, മുമ്പത്തെ എല്ലാ ഉദാഹരണങ്ങളിലെയും വ്യത്യാസമായി, നമ്മുടെ നെറ്റ്‌വർക്ക് ഇപ്പോൾ രണ്ട് പാരാമീറ്ററുകൾ സ്വീകരിക്കുന്നു: ഡാറ്റ വെക്ടറും ഓഫ്‌സെറ്റ് വെക്ടറും, ഇവയുടെ വലിപ്പം വ്യത്യസ്തമാണ്. അതുപോലെ, നമ്മുടെ ഡാറ്റ ലോഡറും 2-ന്റെ പകരം 3 മൂല്യങ്ങൾ നൽകുന്നു: ടെക്സ്റ്റും ഓഫ്‌സെറ്റും വെക്ടറുകളും ഫീച്ചറുകളായി നൽകുന്നു. അതിനാൽ, അത് പരിഗണിച്ച് നമ്മുടെ ട്രെയിനിംഗ് ഫംഗ്ഷൻ ചെറിയ മാറ്റം വരുത്തേണ്ടതുണ്ട്:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## സിമാന്റിക് എംബെഡ്ഡിംഗുകൾ: Word2Vec

മുൻ ഉദാഹരണത്തിൽ, മോഡൽ എംബെഡ്ഡിംഗ് ലെയർ വാക്കുകളെ വെക്ടർ പ്രതിനിധാനമായി മാപ്പ് ചെയ്യാൻ പഠിച്ചു, എന്നാൽ ഈ പ്രതിനിധാനം വളരെ സിമാന്റിക് അർത്ഥം നൽകുന്നില്ല. സമാനമായ വാക്കുകൾ അല്ലെങ്കിൽ പദസമാനാർത്ഥങ്ങൾ ചില വെക്ടർ ദൂരത്തിൽ (ഉദാ. യൂക്ലിഡിയൻ ദൂരം) അടുത്തുള്ള വെക്ടറുകളായി പ്രതിനിധാനം ചെയ്യപ്പെടുന്ന വിധം വെക്ടർ പ്രതിനിധാനം പഠിക്കാനാകും നല്ലത്.

അതിനായി, നമുക്ക് ഒരു വലിയ ടെക്സ്റ്റ് ശേഖരത്തിൽ പ്രത്യേക രീതിയിൽ എംബെഡ്ഡിംഗ് മോഡൽ പ്രീ-ട്രെയിൻ ചെയ്യേണ്ടതുണ്ട്. സിമാന്റിക് എംബെഡ്ഡിംഗുകൾ പരിശീലിപ്പിക്കുന്ന ആദ്യ മാർഗങ്ങളിൽ ഒന്നാണ് [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). ഇത് വാക്കുകളുടെ വിതരണ പ്രതിനിധാനം സൃഷ്ടിക്കാൻ ഉപയോഗിക്കുന്ന രണ്ട് പ്രധാന ആർക്കിടെക്ചറുകളിലാണ് അടിസ്ഥാനമാക്കിയിരിക്കുന്നത്:

 - **കണ്ടിന്യൂവസ് ബാഗ്-ഓഫ്-വേർഡ്സ്** (CBoW) — ഈ ആർക്കിടെക്ചറിൽ, മോഡൽ ചുറ്റുപാടിലുള്ള കോൺടെക്സ്റ്റിൽ നിന്നൊരു വാക്ക് പ്രവചിക്കാൻ പരിശീലിപ്പിക്കുന്നു. n-ഗ്രാം $(W_{-2},W_{-1},W_0,W_1,W_2)$ നൽകിയാൽ, മോഡലിന്റെ ലക്ഷ്യം $(W_{-2},W_{-1},W_1,W_2)$-ൽ നിന്നു $W_0$ പ്രവചിക്കുക എന്നതാണ്.
 - **കണ്ടിന്യൂവസ് സ്കിപ്പ്-ഗ്രാം** CBoW-യുടെ വിപരീതമാണ്. മോഡൽ നിലവിലുള്ള വാക്ക് പ്രവചിക്കാൻ ചുറ്റുപാടിലുള്ള കോൺടെക്സ്റ്റ് വാക്കുകളുടെ വിൻഡോ ഉപയോഗിക്കുന്നു.

CBoW വേഗത്തിൽ പ്രവർത്തിക്കുന്നു, സ്കിപ്പ്-ഗ്രാം മന്ദഗതിയിലാണ്, പക്ഷേ അപൂർവമായ വാക്കുകൾ പ്രതിനിധാനം ചെയ്യുന്നതിൽ മികച്ചതാണ്.

![വാക്കുകളെ വെക്ടറുകളാക്കി മാറ്റാൻ CBoWയും സ്കിപ്പ്-ഗ്രാം ആൽഗോരിതങ്ങളും കാണിക്കുന്ന ചിത്രം.](../../../../../translated_images/ml/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Google News ഡാറ്റാസെറ്റിൽ പ്രീ-ട്രെയിൻ ചെയ്ത word2vec എംബെഡ്ഡിംഗ് പരീക്ഷിക്കാൻ, നാം **gensim** ലൈബ്രറി ഉപയോഗിക്കാം. താഴെ 'neural' എന്ന വാക്കിനോട് ഏറ്റവും സമാനമായ വാക്കുകൾ കാണിക്കുന്നു

> **Note:** വാക്ക് വെക്ടറുകൾ ആദ്യമായി സൃഷ്ടിക്കുമ്പോൾ, ഡൗൺലോഡ് ചെയ്യാൻ കുറച്ച് സമയം എടുക്കാം!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


നാം വാക്കിൽ നിന്നുള്ള വെക്ടർ എംബെഡിംഗുകളും കണക്കാക്കാം, ക്ലാസിഫിക്കേഷൻ മോഡൽ പരിശീലനത്തിൽ ഉപയോഗിക്കാൻ (വെക്ടറിന്റെ ആദ്യ 20 ഘടകങ്ങൾ മാത്രമാണ് വ്യക്തതയ്ക്കായി കാണിക്കുന്നത്):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

സെമാന്റിക്കൽ എംബെഡിംഗുകളുടെ വലിയ ഗുണം എന്തെന്നാൽ, സെമാന്റിക്സ് മാറ്റാൻ വെക്ടർ എൻകോഡിംഗ് മാനിപ്പുലേറ്റ് ചെയ്യാൻ കഴിയും. ഉദാഹരണത്തിന്, *king* ഉം *woman* ഉം എന്ന വാക്കുകളുടെ വെക്ടർ പ്രതിനിധാനത്തിന് όσο അടുത്തുള്ള ഒരു വാക്ക് കണ്ടെത്താൻ, *man* എന്ന വാക്കിൽ നിന്ന് όσο ദൂരമുള്ളതായിരിക്കണം എന്ന് ചോദിക്കാം:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

CBoW ഉം Skip-Grams ഉം “പ്രിഡിക്ടീവ്” embeddings ആണ്, കാരണം അവയ്ക്ക് പ്രാദേശിക കോൺടെക്സ്റ്റുകൾ മാത്രമേ പരിഗണിക്കൂ. Word2Vec ആഗോള കോൺടെക്സ്റ്റിന്റെ പ്രയോജനം ഉപയോഗപ്പെടുത്തുന്നില്ല.

**FastText**, Word2Vec-നെ അടിസ്ഥാനമാക്കി, ഓരോ വാക്കിനും അതിലെ അക്ഷര n-ഗ്രാമുകൾക്കും വെക്ടർ പ്രതിനിധാനങ്ങൾ പഠിക്കുന്നു. ഈ പ്രതിനിധാനങ്ങളുടെ മൂല്യങ്ങൾ ഓരോ പരിശീലന ഘട്ടത്തിലും ശരാശരി എടുത്ത് ഒരു വെക്ടറായി മാറ്റുന്നു. ഇതു പ്രീ-ട്രെയിനിംഗിൽ അധിക കണക്കുകൂട്ടലുകൾ കൂട്ടിച്ചേർക്കുന്നുവെങ്കിലും, വാക്കുകളുടെ സബ്-വേർഡ് വിവരങ്ങൾ എൻകോഡ് ചെയ്യാൻ സാധ്യമാക്കുന്നു.

മറ്റൊരു രീതി, **GloVe**, കോ-ഓക്കറൻസ് മാട്രിക്സ് എന്ന ആശയം ഉപയോഗിച്ച്, കോ-ഓക്കറൻസ് മാട്രിക്സ് ന്യുറൽ രീതികളിലൂടെ കൂടുതൽ പ്രകടനശേഷിയുള്ള, നോൺ-ലീനിയർ വേഡ് വെക്ടറുകളായി വിഭജിക്കുന്നു.

gensim വിവിധ വേഡ് എംബെഡ്ഡിംഗ് മോഡലുകൾ പിന്തുണയ്ക്കുന്നതിനാൽ, FastText, GloVe എന്നിവയിലേക്ക് embeddings മാറ്റി ഉദാഹരണം പരീക്ഷിക്കാം.


## PyTorch-ൽ പ്രീ-ട്രെയിൻ ചെയ്ത എംബെഡിംഗുകൾ ഉപയോഗിക്കൽ

മുകളിൽ നൽകിയ ഉദാഹരണം മാറ്റി, നമ്മുടെ എംബെഡിംഗ് ലെയറിലെ മാട്രിക്സ് സെമാന്റിക്കൽ എംബെഡിംഗുകൾ ഉപയോഗിച്ച് മുൻകൂട്ടി പൂരിപ്പിക്കാം, ഉദാഹരണത്തിന് Word2Vec. പ്രീ-ട്രെയിൻ ചെയ്ത എംബെഡിംഗിന്റെയും നമ്മുടെ ടെക്സ്റ്റ് കോർപസിന്റെയും വാക്കുകളുടെ ലിസ്റ്റുകൾ സാധാരണയായി പൊരുത്തപ്പെടില്ല എന്ന കാര്യം പരിഗണിക്കണം, അതിനാൽ കാണാതിരിക്കുന്ന വാക്കുകൾക്ക് റാൻഡം മൂല്യങ്ങളോടെ വെയ്റ്റുകൾ ആരംഭിക്കും:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


ഇപ്പോൾ നമുക്ക് നമ്മുടെ മോഡൽ പരിശീലിപ്പിക്കാം. മോഡൽ പരിശീലിപ്പിക്കാൻ എടുക്കുന്ന സമയം മുൻവരെയുള്ള ഉദാഹരണത്തേക്കാൾ വളരെ കൂടുതലാണ്, കാരണം എമ്പെഡ്ഡിംഗ് ലെയർ വലുപ്പം കൂടുതലായതിനാൽ, അതിനാൽ പാരാമീറ്ററുകളുടെ എണ്ണം വളരെ കൂടുതലാണ്. കൂടാതെ, ഇതിന്റെ കാരണം, ഓവർഫിറ്റിംഗ് ഒഴിവാക്കാൻ ആഗ്രഹിക്കുന്നുവെങ്കിൽ, നമുക്ക് കൂടുതൽ ഉദാഹരണങ്ങളിൽ മോഡൽ പരിശീലിപ്പിക്കേണ്ടതുണ്ടാകാം.


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

നമ്മുടെ കേസിൽ, വാക്കുകളുടെ വ്യത്യസ്തതകൾ കാരണം കൃത്യതയിൽ വലിയ വർദ്ധനവ് കാണുന്നില്ല. വ്യത്യസ്ത വാക്കുകൾ എന്ന പ്രശ്നം മറികടക്കാൻ, താഴെ പറയുന്ന പരിഹാരങ്ങളിൽ ഒന്നിനെ ഉപയോഗിക്കാം:
* നമ്മുടെ വാക്കുകളുടെ അടിസ്ഥാനത്തിൽ word2vec മോഡൽ വീണ്ടും പരിശീലിപ്പിക്കുക
* മുൻകൂട്ടി പരിശീലിപ്പിച്ച word2vec മോഡലിലെ വാക്കുകൾ ഉപയോഗിച്ച് dataset ലോഡ് ചെയ്യുക. Dataset ലോഡുചെയ്യുമ്പോൾ ഉപയോഗിക്കുന്ന വാക്കുകൾ വ്യക്തമാക്കാം.

അവസാന മാർഗം എളുപ്പമാണ്, പ്രത്യേകിച്ച് PyTorch `torchtext` ഫ്രെയിംവർക്ക് എംബെഡിംഗുകൾക്ക് ഉൾക്കൊള്ളുന്ന പിന്തുണ ഉള്ളതിനാൽ. ഉദാഹരണത്തിന്, GloVe അടിസ്ഥാനമാക്കിയ വാക്കുകൾ ഇങ്ങനെ സൃഷ്ടിക്കാം:


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


ലോഡ് ചെയ്ത വാക്ക് സമാഹാരത്തിന് താഴെ പറയുന്ന അടിസ്ഥാന പ്രവർത്തനങ്ങൾ ഉണ്ട്:
* `vocab.stoi` ഡിക്ഷണറി ഒരു വാക്കിനെ അതിന്റെ ഡിക്ഷണറി ഇൻഡക്സിലേക്ക് മാറ്റാൻ സഹായിക്കുന്നു
* `vocab.itos` അതിന്റെ വിപരീതം ചെയ്യുന്നു - സംഖ്യയെ വാക്കായി മാറ്റുന്നു
* `vocab.vectors` എമ്പെഡിംഗ് വെക്ടറുകളുടെ അറേയാണ്, അതിനാൽ ഒരു വാക്ക് `s` ന്റെ എമ്പെഡിംഗ് ലഭിക്കാൻ `vocab.vectors[vocab.stoi[s]]` ഉപയോഗിക്കണം

ഇവിടെ എമ്പെഡിംഗുകൾ കൈകാര്യം ചെയ്ത് സമവാക്യം **kind-man+woman = queen** കാണിക്കുന്ന ഉദാഹരണം ആണ് (ഇത് പ്രവർത്തിക്കാൻ കോഫിഷ്യന്റ് കുറച്ച് ക്രമീകരിക്കേണ്ടി വന്നു):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

ആ എമ്പെഡിംഗുകൾ ഉപയോഗിച്ച് ക്ലാസിഫയർ പരിശീലിപ്പിക്കാൻ, ആദ്യം നമുക്ക് GloVe വാക്ക് സമാഹാരം ഉപയോഗിച്ച് നമ്മുടെ ഡാറ്റാസെറ്റ് എൻകോഡ് ചെയ്യേണ്ടതുണ്ട്:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

മുകളിൽ കാണിച്ചതുപോലെ, എല്ലാ വെക്ടർ എംബെഡിംഗുകളും `vocab.vectors` മാട്രിക്സിൽ സൂക്ഷിച്ചിരിക്കുന്നു. എളുപ്പത്തിൽ ഈ വെയ്റ്റുകൾ എംബെഡിംഗ് ലെയറിന്റെ വെയ്റ്റുകളിലേക്ക് ലോഡ് ചെയ്യാൻ ഇത് വളരെ എളുപ്പമാക്കുന്നു, സാധാരണ കോപ്പി ചെയ്യലിലൂടെ:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

ഇപ്പോൾ നമുക്ക് നമ്മുടെ മോഡൽ പരിശീലിപ്പിച്ച് മികച്ച ഫലങ്ങൾ ലഭിക്കുന്നുണ്ടോ എന്ന് നോക്കാം:


In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

നമ്മുടെ കൃത്യതയിൽ വലിയ വർദ്ധനവ് കാണാത്തതിന്റെ ഒരു കാരണമാണ് നമ്മുടെ ഡാറ്റാസെറ്റിലെ ചില വാക്കുകൾ പ്രീ-ട്രെയിൻ ചെയ്ത GloVe വാക്കസഞ്ചാരത്തിൽ ഇല്ലാതിരിക്കുക, അതിനാൽ അവ അടിസ്ഥാനപരമായി അവഗണിക്കപ്പെടുന്നു. ഈ പ്രശ്നം മറികടക്കാൻ, നാം നമ്മുടെ ഡാറ്റാസെറ്റിൽ തന്നെ embeddings പരിശീലിപ്പിക്കാം.


## സാന്ദർഭിക എംബെഡിംഗുകൾ

Word2Vec പോലുള്ള പരമ്പരാഗത പ്രീട്രെയിൻഡ് എംബെഡിംഗ് പ്രതിനിധാനങ്ങളുടെ ഒരു പ്രധാന പരിമിതിയാണ് വാക്കിന്റെ അർത്ഥ വ്യത്യാസം തിരിച്ചറിയാനുള്ള പ്രശ്നം. പ്രീട്രെയിൻഡ് എംബെഡിംഗുകൾ വാക്കുകളുടെ ചില അർത്ഥങ്ങൾ സാന്ദർഭികമായി പിടിച്ചുപറ്റാൻ കഴിയുമ്പോഴും, ഒരു വാക്കിന്റെ എല്ലാ സാധ്യതയുള്ള അർത്ഥങ്ങളും ഒരേ എംബെഡിംഗിൽ കോഡ് ചെയ്യപ്പെടുന്നു. ഇത് ഡൗൺസ്ട്രീം മോഡലുകളിൽ പ്രശ്നങ്ങൾ സൃഷ്ടിക്കാം, കാരണം 'play' പോലുള്ള പല വാക്കുകൾ ഉപയോഗിക്കുന്ന സാന്ദർഭം അനുസരിച്ച് വ്യത്യസ്ത അർത്ഥങ്ങൾ ഉണ്ടാകാം.

ഉദാഹരണത്തിന്, 'play' എന്ന വാക്ക് താഴെ കൊടുത്ത രണ്ട് വ്യത്യസ്ത വാചകങ്ങളിൽ വളരെ വ്യത്യസ്ത അർത്ഥം നൽകുന്നു:
- ഞാൻ തിയേറ്ററിൽ ഒരു **play** കാണാൻ പോയി.
- ജോൺ തന്റെ സുഹൃത്തുക്കളുമായി **play** ചെയ്യാൻ ആഗ്രഹിക്കുന്നു.

മുകളിൽ കൊടുത്ത പ്രീട്രെയിൻഡ് എംബെഡിംഗുകൾ ഈ രണ്ട് അർത്ഥങ്ങളും ഒരേ എംബെഡിംഗിൽ പ്രതിനിധാനം ചെയ്യുന്നു. ഈ പരിമിതിയെ മറികടക്കാൻ, വലിയ വാചകശേഖരത്തിൽ പരിശീലിപ്പിച്ച **ഭാഷാ മോഡൽ** അടിസ്ഥാനമാക്കി എംബെഡിംഗുകൾ നിർമ്മിക്കേണ്ടതുണ്ട്, ഇത് വാക്കുകൾ വ്യത്യസ്ത സാന്ദർഭങ്ങളിൽ എങ്ങനെ ചേർക്കാമെന്ന് *അറിയുന്നു*. സാന്ദർഭിക എംബെഡിംഗുകൾ സംബന്ധിച്ച ചർച്ച ഈ ട്യൂട്ടോറിയലിന്റെ പരിധിയിൽ ഇല്ല, പക്ഷേ അടുത്ത യൂണിറ്റിൽ ഭാഷാ മോഡലുകൾക്കുറിച്ച് സംസാരിക്കുമ്പോൾ അവയെക്കുറിച്ച് വീണ്ടും വരാം.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**അസൂയാ**:  
ഈ രേഖ AI വിവർത്തന സേവനം [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. നാം കൃത്യതയ്ക്ക് ശ്രമിച്ചിട്ടുണ്ടെങ്കിലും, സ്വയം പ്രവർത്തിക്കുന്ന വിവർത്തനങ്ങളിൽ പിശകുകൾ അല്ലെങ്കിൽ തെറ്റുകൾ ഉണ്ടാകാമെന്ന് ദയവായി ശ്രദ്ധിക്കുക. അതിന്റെ മാതൃഭാഷയിലുള്ള യഥാർത്ഥ രേഖയാണ് പ്രാമാണികമായ ഉറവിടം എന്ന് പരിഗണിക്കേണ്ടതാണ്. നിർണായകമായ വിവരങ്ങൾക്ക്, പ്രൊഫഷണൽ മനുഷ്യ വിവർത്തനം ശുപാർശ ചെയ്യപ്പെടുന്നു. ഈ വിവർത്തനം ഉപയോഗിക്കുന്നതിൽ നിന്നുണ്ടാകുന്ന ഏതെങ്കിലും തെറ്റിദ്ധാരണകൾക്കോ തെറ്റായ വ്യാഖ്യാനങ്ങൾക്കോ ഞങ്ങൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
